In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings

warnings.filterwarnings('ignore')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load dataset

In [1]:
gender = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [1]:
train.head()

# Cleaning data

In [1]:
def CLEAN(train):
    print(train.groupby(train['Age'].isnull()).median())

    print(train.groupby('Pclass')['Age'].describe())

    for i,j in enumerate(train['Age'].isnull()):
        if train.loc[i,['Pclass']].values ==1:
            train.loc[i,['Age']] =train.loc[i,['Age']].fillna(38.233441)
        elif train.loc[i,['Pclass']].values ==2:
            train.loc[i,['Age']] =train.loc[i,['Age']].fillna(29.877630)
        else:
            train.loc[i,['Age']] =train.loc[i,['Age']].fillna(25.140620)


    train.groupby(train['Cabin'].isnull()).mean()
    train['Cabin'] = np.where(train['Cabin'].isnull(),0,1)

    train['title']= train['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip() )
    li = ['Dr','Master','Miss','Mr','Mrs']

    train['title'] = train['title'].apply(lambda x : x if x in li else 'Other')

    train = train.drop(['Name','Ticket','PassengerId'],axis=1)

    print(train.head())

    from sklearn.preprocessing import LabelEncoder
    li = ['Sex','Embarked','title']

    train = train.dropna()

    for i in li:
      train[i] = LabelEncoder().fit_transform(train[i])

    train['family'] = train['SibSp']+train['Parch']
    train = train.drop(['SibSp','Parch'],axis=1)
    
    return train



In [1]:
train = CLEAN(train)

In [1]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [1]:
support = SVC(kernel='linear',random_state= 42)
random = RandomForestClassifier(random_state=42)

In [1]:
model = [support,random]

In [1]:
test = pd.merge(test,gender,on='PassengerId')

In [1]:
feature = train.drop('Survived',axis=1)
label = train['Survived']




In [1]:
test = CLEAN(test)

In [1]:
testf = test.drop('Survived',axis=1)
testl = test['Survived']




In [1]:
def fitAndScore(feature,label,testf,testl):
    for i in model:
        i.fit(feature,label)

    acc = []
    for i in model:
        acc.append(i.score(testf,testl))
    print(acc)

In [1]:
from imblearn.combine import SMOTEENN
os = SMOTEENN(random_state=42)
feature,label = os.fit_resample(feature,label)

In [1]:
fitAndScore(feature,label,testf,testl)

# Delete outliers

In [1]:
def remove(data):
  Q1 = np.percentile(data, 25, interpolation = 'midpoint')
  
# Third quartile (Q3)
  Q3 = np.percentile(data, 75, interpolation = 'midpoint')
  iqr = Q3 - Q1
  percentile25 = data.quantile(0.25)
  percentile75 = data.quantile(0.75)

  upper_limit = percentile75 + 1.5 * iqr
  lower_limit = percentile25 - 1.5 * iqr
  data = np.where(
    data > upper_limit,upper_limit,
    np.where(
        data < lower_limit,
        lower_limit,
        data
    )
  )
  return data

#train[train['discount_percent'] > upper_limit]
#train[train['discount_percent'] < lower_limit]



In [1]:
plt.figure(figsize=(15,10))
for i in enumerate(train.columns):
  plt.subplot(3,5,i[0]+1)
  sns.boxplot(train[i[1]])


In [1]:
for i in ['Age','Fare']:
    train[i] = remove(train[i])
    test[i] = remove(test[i])

In [1]:
feature = train.drop('Survived',axis=1)
label = train['Survived']

testf = test.drop('Survived',axis=1)
testl = test['Survived']

from imblearn.combine import SMOTEENN
os = SMOTEENN(random_state=42)
feature,label = os.fit_resample(feature,label)

In [1]:
fitAndScore(feature,label,testf,testl)

# Transform

In [1]:
plt.figure(figsize=(15,10))
for i in enumerate(train.columns):
  plt.subplot(3,4,i[0]+1)
  sns.distplot(train[i[1]])

In [1]:
plt.figure(figsize=(15,10))
for i in enumerate(test.columns):
  plt.subplot(3,4,i[0]+1)
  sns.distplot(test[i[1]])

In [1]:
li = ['Age','Fare','family']
from sklearn.preprocessing import  StandardScaler

for i in li:
  train[i] = StandardScaler().fit_transform(train[[i]])
  testf[i] = StandardScaler().fit_transform(testf[[i]])

In [1]:
feature = train.drop('Survived',axis=1)
label = train['Survived']

from imblearn.combine import SMOTEENN
os = SMOTEENN(random_state=42)
feature,label = os.fit_resample(feature,label)

In [1]:
fitAndScore(feature,label,testf,testl)

# GridSearchCV

In [1]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion':['gini','entropy'],
    'n_estimators':[x**2 for x in range(3,10)],
    'n_jobs':[-1,None],
    'random_state':[1,42,None],
    'max_depth': [3,5,None]
}

cv = GridSearchCV(random,params,cv=5)
cv.fit(feature,label)
#print(cv._best_params,cv._best_result)

print(cv.best_params_,cv.best_score_)

bestmodel = cv.best_estimator_
bestmodel.score(testf,testl)